# Tratamento das base de dados das localizações dos radares eletrônicos

### Localização das bases

#### ANTT: https://dados.antt.gov.br/dataset/radar/resource/0e56291a-1092-45d5-b89a-d66b2a89dff4

#### DNIT: https://servicos.dnit.gov.br/multas/informacoes/equipamentos-fiscalizacao

In [87]:
%pip install nbformat
%pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import pandas as pd
import folium

In [89]:
ufs = ['RS', 'SC', 'PR']

In [90]:
df_antt = pd.read_csv("dados/loc_radar/antt.csv", encoding="latin1", sep=";")
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
0,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"483,850",Betim,Principal,Crescente,Ativo,NaN,"-19,959486","-44,085386",80,60
1,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"493,290",Betim,Principal,Crescente,Ativo,NaN,"-19,964349","-44,171924",80,60
2,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"525,350",Brumadinho,Principal,Crescente,Ativo,NaN,"-20,146664","-44,323897",80,60
3,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"528,350",Brumadinho,Principal,Decrescente,Ativo,NaN,"-20,167194","-44,334434",80,60
4,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"568,890",Itaguara,Principal,Decrescente,Ativo,NaN,"-20,410679","-44,526628",80,60


In [91]:
df_antt = df_antt[df_antt['uf'].isin(ufs)]

In [92]:
df_antt.shape

(207, 15)

In [93]:
print(df_antt.columns)

Index(['concessionaria', 'ano_do_pnv_snv', 'tipo_de_radar', 'rodovia', 'uf',
       'km_m', 'municipio', 'tipo_pista', 'sentido', 'situacao',
       'data_da_inativacao', 'latitude', 'longitude', 'velocidade_leve',
       'velocidade_pesado'],
      dtype='object')


In [94]:
# Remove linhas sem coordenadas
df = df_antt.dropna(subset=['latitude', 'longitude'])

In [95]:
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
32,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"664,773",Guaratuba,Principal,Crescente,Ativo,NaN,"-25,858200","-48,951200",60,60
33,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"667,013",Guaratuba,Principal,Crescente,Ativo,NaN,"-25,863700","-48,934800",60,60
34,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"668,334",Guaratuba,Principal,Crescente,Ativo,NaN,"-25,869300","-48,934100",60,60
35,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"668,880",Guaratuba,Principal,Decrescente,Ativo,NaN,"-25,874900","-48,936300",60,60
36,AUTOPISTA LITORAL SUL,2009,Redutor,BR-101,SC,"48,770",Joinville,Principal,Crescente,Ativo,NaN,"-26,380200","-48,851600",80,80


In [96]:
df_antt['latitude'] = df_antt['latitude'].str.replace(',', '.')
df_antt['longitude'] = df_antt['longitude'].str.replace(',', '.')

df_antt['latitude'] = pd.to_numeric(df_antt['latitude'], errors='coerce')
df_antt['longitude'] = pd.to_numeric(df_antt['longitude'], errors='coerce')


In [97]:
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
32,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"664,773",Guaratuba,Principal,Crescente,Ativo,NaN,-25.8582,-48.9512,60,60
33,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"667,013",Guaratuba,Principal,Crescente,Ativo,NaN,-25.8637,-48.9348,60,60
34,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"668,334",Guaratuba,Principal,Crescente,Ativo,NaN,-25.8693,-48.9341,60,60
35,AUTOPISTA LITORAL SUL,2009,Redutor,BR-376,PR,"668,880",Guaratuba,Principal,Decrescente,Ativo,NaN,-25.8749,-48.9363,60,60
36,AUTOPISTA LITORAL SUL,2009,Redutor,BR-101,SC,"48,770",Joinville,Principal,Crescente,Ativo,NaN,-26.3802,-48.8516,80,80


In [98]:
lat_centro = df_antt['latitude'].mean()
lon_centro = df_antt['longitude'].mean()

# Cria mapa com zoom inicial
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=5)

# Adiciona marcadores
for _, row in df_antt.iterrows():
    tooltip = f"""
    <b>Concessionária:</b> {row['concessionaria']}<br>
    <b>Rodovia:</b> {row['rodovia']}<br>
    <b>Município:</b> {row['municipio']}<br>
    <b>Tipo:</b> {row['tipo_de_radar']}<br>
    <b>Vel. leve:</b> {row['velocidade_leve']} km/h
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(mapa)

# Salva e/ou exibe o mapa
mapa.save('radares_antt.html')

In [99]:
df_dnit = pd.read_excel("dados/loc_radar/dnit.xlsx")
df_dnit.head()

,Código do Equipamento,Equipamento,UF,Município,Rodovia,Km,Coordenadas,Registro INMETRO,Nº de Série
0,ROB20040003,Redutor Eletrônico de Velocidade,RO,PIMENTA BUENO,364,"200,150","-61.192778, -11.6525",14781707,FLICD2005A00535
1,ROB20040004,Redutor Eletrônico de Velocidade,RO,PIMENTA BUENO,364,"200,525","-61.19394, -11.64794",14781708,FLICD2005A00536
2,ROB20040008,Redutor Eletrônico de Velocidade,RO,JI-PARANA,364,"343,380","-61.949228, -10.87633",14781722,FLICD2005A00528
3,ROB20040009,Redutor Eletrônico de Velocidade,RO,JI-PARANA,364,"343,360","-61.94878, -10.876266",14781721,FLICD2005A00527
4,ROR20040008,Controlador Eletrônico de Velocidade,RO,JI-PARANA,364,"340,220","-61.930647, -10.896874",14781717,FLISD2005A00524


In [100]:
df_dnit = df_dnit[df_dnit['UF'].isin(ufs)]

In [101]:
# Suponha que a coluna seja chamada 'coordenadas'
df_dnit[['longitude', 'latitude']] = df_dnit['Coordenadas'].str.replace(' ', '').str.split(',', expand=True)
df_dnit.head()

,Código do Equipamento,Equipamento,UF,Município,Rodovia,Km,Coordenadas,Registro INMETRO,Nº de Série,longitude,latitude
237,SCB20010001,Redutor Eletrônico de Velocidade,SC,BOM RETIRO,282,"127,960","-49.487068, -27.803703",14794301,6042,-49.487068,-27.803703
238,SCB20010002,Redutor Eletrônico de Velocidade,SC,BOM RETIRO,282,"128,275","-49.49019, -27.803961",14794302,6043,-49.49019,-27.803961
239,SCB20010003,Redutor Eletrônico de Velocidade,SC,LAGES,282,"215,255","-50.282411, -27.79415",14794315,6046,-50.282411,-27.79415
240,SCB20010007,Redutor Eletrônico de Velocidade,SC,LAGES,282,"218,510","-50.3145, -27.796972",14794310,6052,-50.3145,-27.796972
241,SCB20010008,Redutor Eletrônico de Velocidade,SC,LAGES,282,"218,550","-50.3147, -27.797219",14794313,6053,-50.3147,-27.797219


In [102]:
df_dnit['latitude'] = pd.to_numeric(df_dnit['latitude'], errors='coerce')
df_dnit['longitude'] = pd.to_numeric(df_dnit['longitude'], errors='coerce')
df_dnit.head()

,Código do Equipamento,Equipamento,UF,Município,Rodovia,Km,Coordenadas,Registro INMETRO,Nº de Série,longitude,latitude
237,SCB20010001,Redutor Eletrônico de Velocidade,SC,BOM RETIRO,282,"127,960","-49.487068, -27.803703",14794301,6042,-49.487068,-27.803703
238,SCB20010002,Redutor Eletrônico de Velocidade,SC,BOM RETIRO,282,"128,275","-49.49019, -27.803961",14794302,6043,-49.490190,-27.803961
239,SCB20010003,Redutor Eletrônico de Velocidade,SC,LAGES,282,"215,255","-50.282411, -27.79415",14794315,6046,-50.282411,-27.794150
240,SCB20010007,Redutor Eletrônico de Velocidade,SC,LAGES,282,"218,510","-50.3145, -27.796972",14794310,6052,-50.314500,-27.796972
241,SCB20010008,Redutor Eletrônico de Velocidade,SC,LAGES,282,"218,550","-50.3147, -27.797219",14794313,6053,-50.314700,-27.797219


In [103]:
lat_centro = df_dnit['latitude'].mean()
lon_centro = df_dnit['longitude'].mean()

# Cria mapa com zoom inicial
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=5)

# Adiciona marcadores
for _, row in df_dnit.iterrows():
    tooltip = f"""
    <b>Código do Equipamento:</b> {row['Código do Equipamento']}<br>
    <b>Rodovia:</b> {row['Rodovia']}<br>
    <b>Município:</b> {row['Município']}<br>
    <b>Tipo:</b> {row['Equipamento']}<br>
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(mapa)

# Salva e/ou exibe o mapa
mapa.save('radares_dnit.html')

In [104]:
lat_centro = pd.concat([df_antt['latitude'], df_dnit['latitude']]).mean()
lon_centro = pd.concat([df_antt['longitude'], df_dnit['longitude']]).mean()

# Cria o mapa centralizado
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=5)

# ========== Adiciona marcadores do ANTT (azul) ==========
for _, row in df_antt.iterrows():
    tooltip = f"""
    <b>Concessionária:</b> {row['concessionaria']}<br>
    <b>Rodovia:</b> {row['rodovia']}<br>
    <b>Município:</b> {row['municipio']}<br>
    <b>Tipo:</b> {row['tipo_de_radar']}<br>
    <b>Vel. leve:</b> {row['velocidade_leve']} km/h
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(mapa)

# ========== Adiciona marcadores do DNIT (verde) ==========
for _, row in df_dnit.iterrows():
    tooltip = f"""
    <b>Código do Equipamento:</b> {row['Código do Equipamento']}<br>
    <b>Rodovia:</b> {row['Rodovia']}<br>
    <b>Município:</b> {row['Município']}<br>
    <b>Tipo:</b> {row['Equipamento']}<br>
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='green', icon='camera', prefix='fa')
    ).add_to(mapa)

# Salva o mapa final
mapa.save('radares_combinados.html')

In [105]:
df_2025_ocorrencia = pd.read_csv("dados/2025/datatran2025.csv", encoding="latin1", sep=";")
df_2025_ocorrencia.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,652493,2025-01-01,quarta-feira,06:20:00,SP,116,225,GUARULHOS,Reação tardia ou ineficiente do condutor,Tombamento,...,0,0,1,1,2,"-23,48586772","-46,54075317",SPRF-SP,DEL01-SP,UOP01-DEL01-SP
1,652519,2025-01-01,quarta-feira,07:50:00,CE,116,"546,2",PENAFORTE,Pista esburacada,Colisão frontal,...,0,1,4,1,6,"-7,812288","-39,08333306",SPRF-CE,DEL05-CE,UOP03-DEL05-CE
2,652522,2025-01-01,quarta-feira,08:45:00,PR,369,"88,2",CORNELIO PROCOPIO,Reação tardia ou ineficiente do condutor,Colisão traseira,...,0,2,0,3,2,"-23,182565","-50,637228",SPRF-PR,DEL07-PR,UOP05-DEL07-PR
3,652544,2025-01-01,quarta-feira,11:00:00,PR,116,74,CAMPINA GRANDE DO SUL,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,...,0,4,0,1,2,"-25,36517687","-49,04223028",SPRF-PR,DEL01-PR,UOP02-DEL01-PR
4,652549,2025-01-01,quarta-feira,09:30:00,MG,251,471,FRANCISCO SA,Velocidade Incompatível,Colisão frontal,...,1,1,2,2,4,"-16,46801304","-43,43121303",SPRF-MG,DEL12-MG,UOP01-DEL12-MG


In [106]:
df_2025_ocorrencia = df_2025_ocorrencia[df_2025_ocorrencia['uf'].isin(ufs)]
df_2025_ocorrencia = df_2025_ocorrencia[df_2025_ocorrencia['mortos'] >= 1]


In [107]:
df_2025_ocorrencia['latitude'] = df_2025_ocorrencia['latitude'].str.replace(',', '.')
df_2025_ocorrencia['longitude'] = df_2025_ocorrencia['longitude'].str.replace(',', '.')

df_2025_ocorrencia['latitude'] = pd.to_numeric(df_2025_ocorrencia['latitude'], errors='coerce')
df_2025_ocorrencia['longitude'] = pd.to_numeric(df_2025_ocorrencia['longitude'], errors='coerce')
df_2025_ocorrencia.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
18,652720,2025-01-02,quinta-feira,06:30:00,PR,163,"137,7",CAPITAO LEONIDAS MARQUES,Velocidade Incompatível,Colisão com objeto,...,0,1,2,0,4,-25.521637,-53.594386,SPRF-PR,DEL05-PR,UOP05-DEL05-PR
36,652842,2025-01-02,quinta-feira,17:15:00,PR,369,"413,2",JURANDA,Demais falhas mecânicas ou elétricas,Colisão frontal,...,3,2,1,3,4,-24.452822,-52.848634,SPRF-PR,DEL04-PR,UOP03-DEL04-PR
56,652942,2025-01-03,sexta-feira,10:50:00,SC,101,"40,8",JOINVILLE,Estacionar ou parar em local proibido,Colisão traseira,...,0,2,4,0,6,-26.314791,-48.873358,SPRF-SC,DEL03-SC,UOP01-DEL03-SC
79,653062,2025-01-03,sexta-feira,22:45:00,RS,158,"419,5",SAO GABRIEL,Ausência de reação do condutor,Colisão traseira,...,1,1,1,1,3,-30.039933,-54.591801,SPRF-RS,DEL11-RS,UOP02-DEL11-RS
116,653355,2025-01-05,domingo,15:50:00,SC,470,"80,2",RODEIO,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,...,0,1,2,1,4,-26.948239,-49.318551,SPRF-SC,DEL04-SC,UOP01-DEL04-SC


In [ ]:
import folium
import pandas as pd
from folium import Map, Marker, Icon
from folium.plugins import FloatImage
from branca.element import Template, MacroElement

# Cálculo do centro do mapa com base em todos os dados
lat_centro = pd.concat([
    df_antt['latitude'],
    df_dnit['latitude'],
    df_2025_ocorrencia['latitude']
]).mean()

lon_centro = pd.concat([
    df_antt['longitude'],
    df_dnit['longitude'],
    df_2025_ocorrencia['longitude']
]).mean()

# Cria o mapa
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=5)

# ======== Marcadores ANTT (azul) ========
for _, row in df_antt.iterrows():
    tooltip = f"""
    <b>Concessionária:</b> {row['concessionaria']}<br>
    <b>Rodovia:</b> {row['rodovia']}<br>
    <b>Município:</b> {row['municipio']}<br>
    <b>Tipo:</b> {row['tipo_de_radar']}<br>
    <b>Vel. leve:</b> {row['velocidade_leve']} km/h
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(mapa)

# ======== Marcadores DNIT (verde) ========
for _, row in df_dnit.iterrows():
    tooltip = f"""
    <b>Código do Equipamento:</b> {row['Código do Equipamento']}<br>
    <b>Rodovia:</b> {row['Rodovia']}<br>
    <b>Município:</b> {row['Município']}<br>
    <b>Tipo:</b> {row['Equipamento']}<br>
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='green', icon='camera', prefix='fa')
    ).add_to(mapa)

# ======== Marcadores Ocorrência 2025 (vermelho) ========
for _, row in df_2025_ocorrencia.iterrows():
    tooltip = f"""
    <b>Data:</b> {row.get('data_inversa', 'N/A')}<br>
    <b>Rodovia:</b> {row.get('br', 'N/A')}<br>
    <b>Município:</b> {row.get('municipio', 'N/A')}<br>
    <b>Tipo:</b> {row.get('classificacao_acidente', 'N/A')}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='red', icon='exclamation-triangle', prefix='fa')
    ).add_to(mapa)

# Salvar mapa final
mapa.save('mapa_completo_radares_ocorrencias.html')


TemplateSyntaxError: expected token '(', got 'end of statement block'